Se desarrolla un modelo de recomendación de películas basado en el contenido.

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [ ]:
# importo set limpio 
columnas = ['title', 'genres', 'id', 'release_year', 'runtime', 'popularity', 'cast', 'crew']
df_movies = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv", usecols = columnas)

In [ ]:
#me quedo con las películas con mayor popularidad y menor antiguedad
columnas = ['popularity', 'release_year']  
df_movies = df_movies.sort_values(by=columnas, ascending=False)

In [ ]:
# elimino cortos y series
df_movies = df_movies[(df_movies['runtime'] == 0.0) | (df_movies['runtime']>40)]

In [ ]:
# limpio los campos que se van a utiliza en el modelo
df_movies['genres'] = df_movies['genres'].str.replace("[\[\]',]", "", regex=True)
df_movies['cast'] = df_movies['cast'].str.replace("[\[\]',]", "", regex=True)
df_movies['crew'] = df_movies['crew'].str.replace("[\[\]',]", "", regex=True)



REDUZCO LA CANTIDAD DE REGISTROS




In [ ]:
df_movies = df_movies.head(6000)

In [ ]:
#agrego indice
df_movies.reset_index(drop=True, inplace=True)
df_movies.reset_index(inplace=True)



CONSTRUCCION DEL MODELO




In [ ]:
# features elegidos
selected_features = ['cast','title', 'crew', 'genres']
print(selected_features)

In [ ]:
# reemplazo de nulos
for feature in selected_features:
  df_movies[feature] = df_movies[feature].fillna('')

In [ ]:
#combino el contenido de los features elegidos
combined_features = df_movies['cast']+ ' ' + df_movies['title']+ ' ' + df_movies['genres']+ ' ' + df_movies['crew']

In [ ]:
print(combined_features)

In [ ]:
# creación de vectores con los campos elegidos
vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

In [ ]:
# se calcula la similitud entre las películas con la similitud del coseno
similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

#### busco similitud por nombre

In [ ]:
#pelicula elegida
titulo_pelicula= "The Terminator"


In [ ]:
# se guardan todos los títulos en una variable
list_of_all_titles = df_movies['title'].tolist()
print(list_of_all_titles)

In [ ]:
# se buscan títulos parecidos al target, para determinar si se encuentra
find_close_match = difflib.get_close_matches(titulo_pelicula, list_of_all_titles)


In [ ]:
# se busca el que tenga mayor coincidencia
close_match = find_close_match[0]

In [ ]:
# se accede al índice 
index_of_the_movie = df_movies[df_movies.title == close_match]['index'].values[0]

In [ ]:
# se guardan todos los puntajes, en un tupla commpuesta por el indice de cada pelicula, y el puntaje de la similitud
similarity_score = list(enumerate(similarity[index_of_the_movie]))


In [ ]:
len(similarity_score)

In [ ]:
# se ordenan de mayor similitud a menor
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

In [ ]:
# se arrojan como resultado, las primeras cinco películas, que son las que tienen mayor similitud en base al contenido, definido a partir del
#géneo, el director y los actores

print('Peliculas sugeridas : \n')

i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df_movies[df_movies.index==index]['title'].values[0]
  if (i<6):
    print(i, '.',title_from_index)
    i+=1